In [1]:
%run common.py

Planning start time: Jul 15, 04:30 (1468546244)


In [2]:
st_name = 'ИРКУТСК-СОРТИРОВОЧНЫЙ'

In [3]:
d = datetime.datetime.fromtimestamp(current_time - 18 * 3600)
day_start = int(datetime.datetime.timestamp(datetime.datetime(d.year, d.month, d.day, 18, 0, 0))) + 24 * 3600
print(time.ctime(current_time))
print(time.ctime(day_start))

Fri Jul 15 04:30:44 2016
Fri Jul 15 18:00:00 2016


In [4]:
team_plan['hour_start'] = team_plan.time_start.apply(lambda x: 3600 * int(x / 3600))
team_plan['hour_start_norm'] = team_plan.hour_start.apply(lambda x: time.strftime(time_format, time.localtime(x)))
team_plan['depot_name'] = team_plan.depot.map(st_names.name)
team_plan.loc[team_plan.depot_name.isnull(), 'depot_name'] = team_plan.loc[team_plan.depot_name.isnull(), 'depot']
team_plan['depot_name'].fillna(0, inplace=True)
team_plan['type'] = team_plan.team.apply(lambda x: int(str(x)[0]))
fake_depots = team_plan[(team_plan.state.isin([0, 1])) & (team_plan.type == 7)].drop_duplicates('team')[['team', 'st_from_name']].set_index('team')
team_plan.loc[team_plan.type == 7, 'depot_name'] = team_plan.team.map(fake_depots.st_from_name)

In [5]:
# Total team depart
team_cols = ['team', 'st_from_name', 'st_to_name', 'time_start_norm', 'hour_start_norm', 'state', 'loco', 'depot_name']
a = team_plan[(team_plan.st_from_name == st_name) 
              & (team_plan.state.isin([0, 1]))
              & (team_plan.time_start >= current_time)
              & (team_plan.time_start < current_time + 24 * 3600)].sort_values('time_start')
b = a.groupby(['st_to_name', 'hour_start_norm']).team.count().to_frame().reset_index()
c = b.pivot('st_to_name', 'hour_start_norm', 'team').fillna(0)
print(c.sum(axis = 1))
c

st_to_name
БАТАРЕЙНАЯ         88
ВОЕННЫЙ ГОРОДОК     2
ГОНЧАРОВО          40
dtype: float64


hour_start_norm,"Jul 15, 04:00","Jul 15, 06:00","Jul 15, 07:00","Jul 15, 08:00","Jul 15, 09:00","Jul 15, 10:00","Jul 15, 11:00","Jul 15, 12:00","Jul 15, 13:00","Jul 15, 14:00",...,"Jul 15, 19:00","Jul 15, 20:00","Jul 15, 21:00","Jul 15, 22:00","Jul 15, 23:00","Jul 16, 00:00","Jul 16, 01:00","Jul 16, 02:00","Jul 16, 03:00","Jul 16, 04:00"
st_to_name,,,,,,,,,,,,,,,,,,,,,
БАТАРЕЙНАЯ,1,8,9,6,4,6,4,3,4,4,...,5,4,4,2,1,2,1,3,0,3
ВОЕННЫЙ ГОРОДОК,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
ГОНЧАРОВО,0,2,2,2,3,6,2,2,0,4,...,1,0,2,1,1,0,3,3,0,0


In [7]:
b_dep = a.groupby(['depot_name', 'hour_start_norm']).team.count().to_frame().reset_index()
c_dep = c = b_dep.pivot('depot_name', 'hour_start_norm', 'team').fillna(0)
print(c_dep.sum(axis = 1).sort_values(ascending=False))
c_dep

depot_name
ЗИМА                     61
ИРКУТСК-СОРТИРОВОЧНЫЙ    50
НИЖНЕУДИНСК               9
СЛЮДЯНКА I                7
ЧЕЛУТАЙ                   1
УЛАН-УДЭ                  1
ТУЛУН                     1
dtype: float64


hour_start_norm,"Jul 15, 04:00","Jul 15, 06:00","Jul 15, 07:00","Jul 15, 08:00","Jul 15, 09:00","Jul 15, 10:00","Jul 15, 11:00","Jul 15, 12:00","Jul 15, 13:00","Jul 15, 14:00",...,"Jul 15, 19:00","Jul 15, 20:00","Jul 15, 21:00","Jul 15, 22:00","Jul 15, 23:00","Jul 16, 00:00","Jul 16, 01:00","Jul 16, 02:00","Jul 16, 03:00","Jul 16, 04:00"
depot_name,,,,,,,,,,,,,,,,,,,,,
ЗИМА,1,7,6,4,3,5,4,1,3,5,...,0,1,1,1,1,2,1,3,0,3
ИРКУТСК-СОРТИРОВОЧНЫЙ,1,3,5,4,4,7,2,4,1,3,...,1,1,3,0,1,0,1,2,0,0
НИЖНЕУДИНСК,0,0,0,0,0,0,0,0,0,0,...,2,2,2,1,0,0,0,0,0,0
СЛЮДЯНКА I,0,0,0,0,0,0,0,0,0,0,...,3,0,0,0,0,0,2,0,0,0
ТУЛУН,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
УЛАН-УДЭ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
ЧЕЛУТАЙ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [8]:
st1 = 'БАТАРЕЙНАЯ'
d = team_plan[(team_plan.st_from_name == st_name) & (team_plan.st_to_name == st1)
           & (team_plan.state.isin([0, 1]))
           & (team_plan.time_start >= day_start)
           & (team_plan.time_start < day_start + 24 * 3600)].sort_values('time_start')
d[d.state == 1].hour_start_norm.value_counts().sort_index()
#d[d.state == 1][team_cols].head(10)

Jul 15, 18:00    3
Jul 15, 19:00    4
Jul 15, 20:00    4
Jul 15, 21:00    4
Jul 15, 22:00    2
Jul 15, 23:00    1
Jul 16, 00:00    2
Jul 16, 01:00    1
Jul 16, 02:00    3
Jul 16, 04:00    4
Jul 16, 05:00    2
Jul 16, 06:00    2
Jul 16, 07:00    4
Jul 16, 08:00    2
Jul 16, 09:00    1
Jul 16, 10:00    4
Jul 16, 12:00    1
Jul 16, 13:00    1
Jul 16, 14:00    3
Jul 16, 17:00    2
Name: hour_start_norm, dtype: int64

In [10]:
team_plan['type'] = team_plan.team.apply(lambda x: int(str(x)[:1]))
print(time.ctime(current_time))
team_plan[(team_plan['type'] == 7) & (team_plan.state.isin([0, 1]) & (team_plan.time_start <= current_time + 30 * 3600))]\
        .drop_duplicates('team').hour_start_norm.value_counts().sort_index()

Fri Jul 15 04:30:44 2016


Jul 15, 04:00     4
Jul 15, 05:00     4
Jul 15, 06:00    39
Jul 15, 07:00    19
Jul 15, 08:00    17
Jul 15, 09:00    21
Jul 15, 10:00    23
Jul 15, 11:00    19
Jul 15, 12:00    12
Jul 15, 13:00    12
Jul 15, 14:00    21
Jul 15, 15:00    22
Jul 15, 16:00    19
Jul 15, 17:00    24
Jul 15, 18:00    24
Jul 15, 19:00    19
Jul 15, 20:00    27
Jul 15, 21:00    24
Jul 15, 22:00    30
Jul 15, 23:00    34
Jul 16, 00:00    48
Jul 16, 01:00    41
Jul 16, 02:00    37
Jul 16, 03:00    39
Jul 16, 04:00    31
Jul 16, 05:00    41
Jul 16, 06:00    23
Jul 16, 07:00    12
Jul 16, 08:00     8
Jul 16, 09:00     6
Jul 16, 10:00     3
Name: hour_start_norm, dtype: int64

In [11]:
st_name = 'ИРКУТСК-СОРТИРОВОЧНЫЙ'
irk_t = team_plan[(team_plan.st_from_name == st_name) & (team_plan.state.isin([0, 1]))
         & (team_plan.time_start >= current_time) & (team_plan.time_start < current_time + 24 * 3600)]
irk = team_plan[(team_plan.team.isin(irk_t.team)) & (team_plan.state.isin([0, 1]))].copy(deep=True)
mins = irk.groupby('team').time_start.min()
irk['end_trip'] = (irk.team != irk.team.shift(-1)) | (irk.state != irk.state.shift(-1)) | (irk.loco != irk.loco.shift(-1))
irk['start_trip'] = (irk.team != irk.team.shift(1)) | (irk.state != irk.state.shift(1)) | (irk.loco != irk.loco.shift(1))
irk[['team', 'start_trip', 'end_trip']]
irk[(irk.start_trip == True) | (irk.end_trip == True)][['team', 'st_from_name', 'st_to_name', 'state', 'loco']]
starts = irk[(irk.start_trip == True)][['team', 'st_from_name', 'time_start', 'time_start_norm', 'state', 'loco']].reset_index()
ends = irk[(irk.end_trip == True)][['st_to_name', 'time_end', 'time_end_norm']].reset_index()
cols = ['team', 'st_from_name', 'st_to_name', 'time_start', 'time_end', 'time_start_norm', 'time_end_norm', 'state', 'loco']
trips = pd.concat([starts, ends], axis=1)[cols]
trips[(trips.st_from_name == st_name)].st_to_name.value_counts()

ЗИМА                     36
СЛЮДЯНКА I               25
УЛАН-УДЭ                  8
НИЖНЕУДИНСК               5
КИТОЙ                     4
СУХОВСКАЯ                 4
МЕГЕТ                     3
ЧЕРЕМХОВО                 2
ИРКУТСК-СОРТИРОВОЧНЫЙ     2
СУХОВСКАЯ-ЮЖНАЯ           2
КАЯ                       1
КИТОЙ-КОМБИНАТСКАЯ        1
БОЛЬШОЙ ЛУГ               1
ЧЕЛУТАЙ                   1
Name: st_to_name, dtype: int64